# AMS Benchmark

Reference:

1. Power Grid Lib - Optimal Power Flow, https://github.com/power-grid-lib/pglib-opf

In [1]:
import platform
import os

from tqdm import tqdm
import numpy as np
import pandas as pd

import andes
import ams

import pypower.api as pyp

import datetime
import time

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-03-26 17:33:52
andes:1.9.1.post24+g7a87ad5d
ams:0.9.5.post1+g114d11e


In [3]:
andes.config_logger(stream_level=50)
ams.config_logger(stream_level=50)

In [4]:
directory = '../../cases/pglib-opf/api/'

# List all files and directories within the specified directory
entries = os.listdir(directory)

# Get a list of files in the directory along with their sizes
files_with_sizes = [(file, os.path.getsize(os.path.join(directory, file))) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]

# Sort the list of files by size (the second element of the tuple)
sorted_files_with_sizes = sorted(files_with_sizes, key=lambda x: x[1])

# If you just want a list of filenames sorted by size
sorted_files0 = [file for file, size in sorted_files_with_sizes]

files_to_remove = ['.DS_Store']

sorted_files = [file for file in sorted_files0 if file not in files_to_remove]

In [5]:
n_cases = len(sorted_files)

res_cols = ['case', 'a_diff_max',
            'pi_diff_max', 'obj_diff',
            't_ami', 't_am', 't_pp',
            'c_am', 'c_pp']
res = pd.DataFrame(columns=res_cols)

n_cases = len(sorted_files)
n_cases = 38  # test partial cases

for case_i in tqdm(range(n_cases)):
    opf_case = sorted_files[case_i]
    sp = ams.load(directory + opf_case,
                  setup=True, no_output=True, default_config=True)
    # skip distribution cases
    if sp.Slack.n == 0:
        continue
    t_ami = time.time()
    sp.DCOPF.init()
    s_adi = time.time() - t_ami

    t_am = time.time()
    sp.DCOPF.run(solver='GUROBI', reoptimize=True, ignore_dpp=True)
    s_am = time.time() - t_am

    # --- PYPOWER ---
    ppc = ams.io.pypower.system2ppc(sp)
    ppopt = pyp.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1, OPF_ALG_DC=200)
    t_pp = time.time()
    ppc_sol = pyp.rundcopf(ppc, ppopt)
    s_pp = time.time() - t_pp

    if (sp.DCOPF.exit_code == 0) & (ppc_sol['success']):
        a_am = sp.DCOPF.aBus.v - sp.DCOPF.aBus.v[0]
        pi_am = sp.DCOPF.pi.v
        a_pp = (ppc_sol['bus'][:, 8] - ppc_sol['bus'][0, 8]) * andes.shared.deg2rad
        pi_pp = ppc_sol['bus'][:, 13] * ppc_sol['baseMVA']

        a_diff = np.abs(a_am - a_pp)
        pi_diff = np.abs(pi_am - pi_pp)
        obj_diff = sp.DCOPF.obj.v - ppc_sol['f']
    else:
        a_diff = np.array([-1])
        pi_diff = np.array([-1])
        obj_diff = -1

    row_res = {'case': opf_case.split('.')[0],
               'a_diff_max': np.max(a_diff),
               'pi_diff_max': np.max(pi_diff),
               'obj_diff': obj_diff,
               't_ami': s_adi, 't_am': s_am, 't_pp': s_pp,
               'c_am': int(1 - sp.DCOPF.exit_code),
               'c_pp': int(ppc_sol['success'])}
    res.loc[case_i] = row_res

  0%|          | 0/38 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-17


100%|██████████| 38/38 [00:55<00:00,  1.46s/it]


In [6]:
res.round(6)

,case,a_diff_max,pi_diff_max,obj_diff,t_ami,t_am,t_pp,c_am,c_pp
0,pglib_opf_case3_lmbd__api,0.000000,0.000000,-0.000000,0.010671,0.033149,0.017773,1,1
1,pglib_opf_case5_pjm__api,0.000000,0.000001,-0.000001,0.005344,0.012719,0.025337,1,1
2,pglib_opf_case14_ieee__api,0.000000,0.000000,-0.000000,0.005698,0.006741,0.014458,1,1
3,pglib_opf_case30_ieee__api,0.000000,0.000000,-0.000000,0.005959,0.007824,0.015350,1,1
4,pglib_opf_case30_as__api,0.000000,0.000002,-0.000000,0.005673,0.011154,0.023493,1,1
5,pglib_opf_case57_ieee__api,0.000000,0.000000,-0.000000,0.006291,0.065618,0.024572,1,1
6,pglib_opf_case60_c__api,0.055586,0.000175,-0.000001,0.006584,0.008433,0.066402,1,1
7,pglib_opf_case24_ieee_rts__api,0.000000,0.000000,-0.000000,0.006507,0.007734,0.031750,1,1
8,pglib_opf_case89_pegase__api,0.000000,0.000000,-0.000000,0.006358,0.014423,0.045525,1,1
9,pglib_opf_case162_ieee_dtc__api,0.000000,0.000000,-0.000000,0.007021,0.013972,0.101869,1,1
